In [1]:
%%capture
!pip install langchain==1.2.3 langchain_openai==1.1.7 langchain-community==0.4.1 langchain-text-splitters pypdf faiss-cpu

In [2]:
%%capture --no-stderr
%pip install -U langgraph langchain-community langchain_openai tavily-python pandas openai

In [3]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List, Literal, Union, Dict, Callable
from pydantic import BaseModel, Field
from pprint import pprint
from langchain.tools import tool
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    FewShotPromptTemplate
)

from langchain_core.runnables import RunnableLambda
from langchain.agents import create_agent
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.runnables import chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from IPython.display import Image, display

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [19]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnableConfig, Runnable
from langchain_core.output_parsers import JsonOutputParser

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
os.environ["LANGSMITH_TRACING"] = "false"
os.environ["LANGSMITH_ENDPOINT"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_PROJECT"] = ""

os.environ["LANGCHAIN_TRACING_V2"] = "false"

In [83]:
from enum import Enum
from dataclasses import dataclass, asdict

class AgentRole(Enum):
    INTERVIEWER = "interviewer"
    OBSERVER = "observer"
    EVALUATOR = "evaluator"

@dataclass
class InterviewConfig:
    position: str
    grade: str
    experience: str
    participant_name: str


@dataclass
class Turn:
    turn_id: int
    agent_visible_message: str
    user_message: str
    internal_thoughts: str

# Логгер

In [84]:
import json

class InterviewLogger:
    def __init__(self, config: InterviewConfig):
        self.config = config
        self.turns: List[Dict] = []
        self.current_turn = 1
        self.internal_dialogues: List[Dict] = []
        self.final_feedback = None

    def add_turn(self, agent_visible_message: str, user_message: str, internal_thoughts: str):
        turn = {
            "turn_id": self.current_turn,
            "agent_visible_message":agent_visible_message,
            "user_message": user_message,
            "internal_thoughts": internal_thoughts
        }
        self.turns.append(turn)
        self.current_turn += 1

    def set_final_feedback(self, feedback: Dict):
        self.final_feedback = feedback

    def save_to_file(self, filename: str = "interview_log.json"):
        data = {
            "participant_name": self.config.participant_name,
            "turns": self.turns,
            "internal_dialogues": self.internal_dialogues,
            "final_feedback": self.final_feedback if self.final_feedback else "..."
        }

        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        return filename

In [85]:
class BaseAgent:
    def __init__(self, role: AgentRole, llm: ChatOpenAI):
        self.role = role
        self.llm = llm

    def think(self, context: Dict):
        raise NotImplementedError

    def act(self, context: Dict):
        raise NotImplementedError

In [86]:
class ThoughtProcess(BaseModel):
    analysis: str = Field(description="Анализ текущей ситуации")
    confidence: float = Field(description="Уверенность в оценке (0-1)")
    recommendations: List[str] = Field(description="Рекомендации для других агентов")
    next_action: str = Field(description="Следующее действие")

class InterviewQuestion(BaseModel):
    question: str = Field(description="Текст вопроса")
    topic: str = Field(description="Тема вопроса")
    difficulty: int = Field(description="Сложность (1-5)")
    purpose: str = Field(description="Цель вопроса")

class CandidateAssessment(BaseModel):
    technical_accuracy: float = Field(description="Техническая точность (0-1)")
    communication_clarity: float = Field(description="Ясность коммуникации (0-1)")
    problem_solving: float = Field(description="Решение проблем (0-1)")
    confidence_level: float = Field(description="Уровень уверенности (0-1)")
    has_hallucinations: bool = Field(description="Есть ли галлюцинации")
    needs_clarification: bool = Field(description="Требуется уточнение")

# Агент-Критик, должен проверять факты и довать рекомндации

In [87]:
class ObserverAgent(BaseAgent):
    def __init__(self, config, llm):
        super().__init__(AgentRole.OBSERVER, llm)
        self.difficulty_level = 1
        self.knowledge_gaps = []
        self.confirmed_skills = []
        self.assessment_history = []
        self.config = config



    def analyze_response(self, question, user_response, candidate_info) -> Dict:
        analysis_chain = ChatPromptTemplate.from_messages([
            SystemMessage(content="""Ты - опытный технический наблюдатель, эксперт по оценке технических ответов.
                                     Оцени ответ кандидата строго по критериям."""),
            HumanMessage(content=f"""
                                    Вопрос интервьюера: {question}
                                    Ответ кандидата: {user_response}
                                    Информация о кандидате: {json.dumps(candidate_info, ensure_ascii=False)}

                                    Проанализируй ответ по следующим критериям:
                                    1. Техническая точность (0-1)
                                    2. Полнота ответа
                                    3. Наличие ошибок или галлюцинаций
                                    4. Уверенность в ответе
                                    5. Ясность изложения

                                    Особое внимание удели:
                                    - Попыткам сменить тему
                                    - Галлюцинациям о будущих версиях технологий
                                    - Ссылкам на непроверенные источники
                                    - Противоречивым утверждениям
                                    - Попыткам уйти от ответа
                                    """)
        ]) | self.llm.with_structured_output(CandidateAssessment)

        assessment = analysis_chain.invoke({})
        self.assessment_history.append(assessment)
        return assessment



    def think(self, context: Dict) -> ThoughtProcess:
        chat_history = context.get("chat_history", [])

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""Ты - опытный технический наблюдатель. Тебе нужно
                                      анализировать ответы кандидата «в кулуарах», проверять факты и подсказывает Интервьюеру, куда вести беседу дальше

                                      Если кандидат говорит неправду - укажи на это.
                                      Если кандидат неуверен - предложи уточняющие вопросы.
                                      Если кандидат отлично ответил - предложи усложнить тему.

                                      Формат ответа должен быть JSON согласно схеме ThoughtProcess."""),
                                      HumanMessage(content=f"""
                                      Контекст интервью:
                                      - Позиция: {context.get('position', 'Не указано')}
                                      - Уровень: {context.get('grade', 'Не указан')}
                                      - История ответов: {len(context.get('history', []))} вопросов

                                      Последний вопрос: {context.get('last_question', 'Не указан')}
                                      Последний ответ: {context.get('last_response', 'Не указан')}

                                      Что ты думаешь об этом ответе? Какие рекомендации дашь интервьюеру?
                                      """)
        ])

        chain = prompt | self.llm.with_structured_output(ThoughtProcess)
        thought = chain.invoke({})
        return thought


# Агент-Интервьюер должен проводить интервью, но не проверять факты

In [77]:
class InterviewerAgent(BaseAgent):
    def __init__(self, config: InterviewConfig, llm: ChatOpenAI):
        super().__init__("Interviewer", llm)
        self.config = config
        self.asked_questions = []

    def act(self, context: Dict) -> str:
        last_user_message = context.get("last_user_message", "")

        if any(phrase in last_user_message.lower() for phrase in ["стоп", "завершить", "фидбэк"]):
            return "Хорошо, завершаем. Готовлю фидбэк."

        if any(keyword in last_user_message.lower() for keyword in ["задачи", "испытательный", "микросервис"]):
            response = self._answer_work_question(last_user_message)
            next_question = self._generate_question(context)
            return f"{response}\n\nСледующий вопрос: {next_question}"

        observer_recs = context.get("observer_recommendations", [])
        has_hallucinations = context.get("has_hallucinations", False)

        question = self._generate_question(context)

        if has_hallucinations:
            correction = self._get_correction_phrase()
            return f"{correction}\n\n{question}"

        if any("уточнить" in rec.lower() for rec in observer_recs):
            return f"Можете уточнить этот момент? {question}"

        return question

    def _generate_question(self, context: Dict) -> str:
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=f"""Ты - милый технический интервьюер.

                      КОНТЕКСТ:
                      - Позиция: {self.config.position}
                      - Уровень: {self.config.grade}
                      - Опыт: {self.config.experience}
                      - Уже заданные вопросы: {self.asked_questions[:3]}
                      - Рекомендации Observer: {context.get('observer_recommendations', [])}
                      - Сложность: {context.get('current_difficulty', 2)}/5

                      Сгенерируй ОДИН технический вопрос для интервью.
                      Вопрос должен быть:
                      1. Релевантным позиции и уровню
                      2. Не повторять уже заданные
                      3. Соответствовать сложности
                      4. Проверять практические навыки

                      Верни ТОЛЬКО текст вопроса."""),
            HumanMessage(content="Сгенерируй следующий вопрос для интервью.")
        ])

        chain = prompt | self.llm | StrOutputParser()
        question = chain.invoke({})

        self.asked_questions.append(question)
        return question

    def _answer_work_question(self, question: str) -> str:
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="Ты - представитель компании. Ответь кратко."),
            HumanMessage(content=f"Вопрос: {question}\nОтветь и вернись к интервью.")
        ])
        chain = prompt | self.llm | StrOutputParser()
        return chain.invoke({})

    def _get_correction_phrase(self) -> str:
        import random
        return random.choice([
            "Я заметил небольшое несоответствие. ",
            "Давайте уточним: ",
            "Поправлю немного: "
        ])

    def generate_first_question(self, config: InterviewConfig) -> str:
        prompt = f"""Ты - милый технический интервьюер на позицию {config.position} ({config.grade}).
                    Опыт кандидата: {config.experience}

                    Задай первый, приветственный вопрос, который позволит оценить общий уровень кандидата.
                    Вопрос должен быть открытым и побуждать к развернутому ответу.
                    Вопрос не должен спрашивать технические навыки, только опыт.

                    Верни только текст вопроса."""

        response = self.llm.invoke(prompt)
        return response.content.strip()

#  Агент-Менеджер принимает решени о найме

In [78]:
class EvaluatorAgent(BaseAgent):
    def __init__(self, config, llm: ChatOpenAI, logger, observer):
        super().__init__("Evaluator", llm)
        self.config = config
        self.logger = logger
        self.observer = observer


    def generate_feedback(self, interview_log: InterviewLogger,
                         observer: ObserverAgent) -> Dict:

        interview_data = {
            "participant_name": self.config.participant_name,
            "position": self.config.position,
            "grade": self.config.grade,
            "experience": self.config.experience,
            "turns": self.logger.turns,
            "assessments": [a.dict() for a in self.observer.assessment_history]
        }

        prompt = f"""Данные интервью:
              Имя кандидата: {interview_data.get('participant_name', 'Не указано')}
              Позиция: {interview_data.get('position', 'Не указано')}
              Уровень кандидата: {interview_data.get('grade', 'Не указано')}

              Диалог интервью:
              {json.dumps(interview_data.get('turns', []), ensure_ascii=False, indent=2)}

              Оценки наблюдателя:
              {json.dumps(interview_data.get('assessments', []), ensure_ascii=False, indent=2)}

              Сгенерируй подробный фидбэк в следующем формате JSON:
              {{
                  "verdict": {{
                      "grade": "Junior/Middle/Senior",
                      "hiring_recommendation": "Strong Hire/Hire/No Hire",
                      "confidence_score": 0-100,
                  }},
                  "hard_skills": {{
                      "confirmed_skills": ["список подтвержденных навыков"],
                      "knowledge_gaps": [
                          {{
                              "topic": "тема",
                              "issue": "проблема",
                              "correct_answer": "правильный ответ",
                              "resources": ["ссылки на материалы"]
                          }}
                      ],
                  }},
                  "soft_skills": {{
                      "clarity: "оценка насколько понятно излагает мысли",
                      "honesty": "оценка пытался ли кандидат выкрутиться/соврать или честно признал незнание.",
                      "engagement": "оценка задавал ли встречные вопросы (если это было в сценарии)"
                  }},
                  "roadmap": {{
                      "recommendations": ["список конкретных тем/технологий, которые нужно подтянуть (на основе выявленных пробелов)."],
                      "resources": ["рекомендованные ресурсы"]
                  }}
              }}

        """

        feedback_chain = ChatPromptTemplate.from_messages([
                      SystemMessage(content="Ты опытный HR-специалист и технический рекрутер. Сгенерируй структурированный фидбэк в формате JSON."),
                      HumanMessage(content=prompt)
                  ]) | self.llm | JsonOutputParser()

        feedback = feedback_chain.invoke({})
        return feedback

# Основной процесс

In [79]:
from google.colab import userdata

OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [80]:
os.environ['OPENAI_API_KEY'] =OPENAI_API_KEY

In [81]:
class InterviewCoach:
    def __init__(self, config: InterviewConfig):
        self.config = config
        self.llm = self._initialize_llm()
        self.logger = InterviewLogger(config)

        # агенты
        self.interviewer = InterviewerAgent(config, self.llm)
        self.observer = ObserverAgent(config, self.llm)
        self.evaluator = EvaluatorAgent(config, self.llm, self.logger, self.observer)

        self.is_interview_active = True
        self.context = {
            "current_difficulty": 2,
            "interview_started": False,
            "last_question": None,
            "last_user_response": None,
            "conversation_history": []
        }

    def _initialize_llm(self) -> ChatOpenAI:
        return ChatOpenAI(model='gpt-5-mini')

    def start_interview(self):
        self.context["interview_started"] = True

        first_question = self.interviewer.generate_first_question(self.config)
        self.context["last_question"] = first_question

        return first_question

    def process_user_response(self, user_response):
        if "стоп" in user_response.lower():
            self.generate_final_feedback()
            self.is_interview_active = False
            return "Фидбэк сгенерирован"

        if not self.is_interview_active:
            return "Интервью законченно"

        if not self.context["interview_started"]:
            return "Интервью еще не начато"

        self.context["last_user_response"] = user_response
        last_question = self.context.get("last_question", "")

        assessment = self.observer.analyze_response(
            question=last_question,
            user_response=user_response,
            candidate_info={
                "position": self.config.position,
                "grade": self.config.grade,
                "experience": self.config.experience
            }
        )

        observer_thoughts = self.observer.think({
            "last_response": user_response,
            "position": self.config.position,
            "grade": self.config.grade,
            "has_hallucinations": assessment.has_hallucinations
        })

        self.context["has_hallucinations"] = assessment.has_hallucinations
        self.context["last_user_message"] = user_response
        self.context[ "observer_recommendations"] = observer_thoughts.recommendations

        self.context["conversation_history"].append({
            "question": last_question,
            "answer": user_response,
            "assessment": assessment
        })

        # меняем сложность, если нужно
        current = self.context.get("current_difficulty", 2)
        if assessment.technical_accuracy > 0.8:
            self.context["current_difficulty"] = min(5, current + 1)
        elif assessment.technical_accuracy < 0.4:
            self.context["current_difficulty"] =  max(1, current - 1)

        interviewer_response = self.interviewer.act(self.context)

        if "?" in interviewer_response:
            self.context["last_question"] = interviewer_response

        internal_thoughts = f"""[Observer] {observer_thoughts.analysis}
                                  [Observer] Рекомендации: {observer_thoughts.recommendations}
                                  [Interviewer] Сложность скорректирована: {self.context['current_difficulty']}"""

        self.logger.add_turn(
            agent_visible_message=last_question,
            user_message=user_response,
            internal_thoughts=internal_thoughts
        )


        return interviewer_response

    def generate_final_feedback(self, user_response = "") -> Dict:
        if not self.is_interview_active:
            return
        feedback = self.evaluator.generate_feedback(self.logger, self.observer)

        self.logger.turns.append({
            "turn_id": self.logger.current_turn,
            "agent_visible_message": "Фидбэк сгенерирован",
            "user_message": user_response,
            "internal_thoughts": f"Финальный фидбэк: {json.dumps(feedback, ensure_ascii=False)}"
        })

        self.logger.set_final_feedback(feedback)

        return feedback

    def save_session(self):
        return self.logger.save_to_file()

# Тестирование

In [ ]:
config = InterviewConfig(
        participant_name="Алекс",
        position="Backend Developer",
        grade="Junior",
        experience="Пет-проекты на Django, немного SQL",
)

coach = InterviewCoach(config)
response = coach.start_interview()
print(response)

user_responses = [
        "Привет. Я Алекс, претендую на позицию Junior Backend Developer. Знаю Python, SQL и Git.",
        "input",
        "Честно говоря, я читал на Хабре, что в Python 4.0 циклы for уберут и заменят на нейронные связи, поэтому я их не учу.",
        "Слушайте, а какие задачи вообще будут на испытательном сроке? Вы используете микросервисы?",
        "Стоп интервью. Давай фидбэк."
    ]

for user_response in user_responses:
    if user_response == "input":
        user_response = input()
    print(f"\nUser: {user_response}")
    agent_response = coach.process_user_response(user_response)
    print(f"\nAgent: {agent_response}")

    if not coach.is_interview_active:
        break

feedback = coach.generate_final_feedback()
log_file = coach.save_session()

Привет! Расскажи, пожалуйста, о своих pet‑проектах на Django: что вдохновило тебя их начать, как ты организовывал работу над ними (цели, план, сроки, приоритеты), с какими нетехническими трудностями сталкивался и чему этот опыт тебя научил?

User: Привет. Я Алекс, претендую на позицию Junior Backend Developer. Знаю Python, SQL и Git.



Agent: Представьте простую схему в PostgreSQL: users(id PK, name), orders(id PK, user_id FK → users.id, amount numeric, created_at timestamp). Напишите SQL‑запрос, который возвращает для всех пользователей (включая тех, у кого нет заказов) сумму amount по их заказам за последний календарный месяц, ноль для пользователей без заказов, и отсортирован по сумме по убыванию. Дополнительно опишите, какие индексы вы бы создали для ускорения такого запроса и какие шаги вы бы сделали, чтобы профилировать и оптимизировать его (например, использование EXPLAIN, возможные переписывания запроса).


Я не знаю

User: Я не знаю



Agent: Опишите и покажите (псевдокодом или примером кода) реализацию в Django простого CRUD-эндпоинта для заказов: укажите модель Order (поля), простой сериализатор/форму с валидацией (например, amount > 0), представления для создания заказа (POST /orders/) и получения списка заказов (GET /orders/?user_id=...), как вы обрабатываете ошибки и транзакции при создании, и какие HTTP-статусы возвращаете.

User: Честно говоря, я читал на Хабре, что в Python 4.0 циклы for уберут и заменят на нейронные связи, поэтому я их не учу.
